In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.info()

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [8]:
#drop id columns, ones that are not necessary

data = [train, test]

df = pd.concat(data, ignore_index = True)

cols = ['ID','Customer_ID','Month','Name','SSN','Monthly_Inhand_Salary','Num_Bank_Accounts','Num_Credit_Card',
         'Interest_Rate','Num_of_Loan','Type_of_Loan','Changed_Credit_Limit','Num_Credit_Inquiries','Credit_Mix',
         'Credit_Utilization_Ratio','Amount_invested_monthly']

df = df.drop(cols, axis = 1)
df

,Age,Occupation,Annual_Income,Delay_from_due_date,Num_of_Delayed_Payment,Outstanding_Debt,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Payment_Behaviour,Monthly_Balance,Credit_Score
0,23,Scientist,19114.12,3,7,809.98,22 Years and 1 Months,No,49.574949,High_spent_Small_value_payments,312.49408867943663,Good
1,23,Scientist,19114.12,-1,NaN,809.98,NaN,No,49.574949,Low_spent_Large_value_payments,284.62916249607184,Good
2,-500,Scientist,19114.12,3,7,809.98,22 Years and 3 Months,No,49.574949,Low_spent_Medium_value_payments,331.2098628537912,Good
3,23,Scientist,19114.12,5,4,809.98,22 Years and 4 Months,No,49.574949,Low_spent_Small_value_payments,223.45130972736786,Good
4,23,Scientist,19114.12,6,NaN,809.98,22 Years and 5 Months,No,49.574949,High_spent_Medium_value_payments,341.48923103222177,Good
...,...,...,...,...,...,...,...,...,...,...,...,...
149995,4975,Architect,20002.88,33,25,3571.7,NaN,Yes,60.964772,Low_spent_Small_value_payments,275.53956951573343,NaN
149996,25,Mechanic,39628.99,20,NaN,502.38,31 Years and 11 Months,NM,35.104023,Low_spent_Small_value_payments,409.39456169535066,NaN
149997,25,Mechanic,39628.99,23,5,502.38,32 Years and 0 Months,No,35.104023,Low_spent_Large_value_payments,349.7263321025098,NaN
149998,25,Mechanic,39628.99,21,6_,502.38,32 Years and 1 Months,No,35.104023,High_spent_Small_value_payments,463.23898098947717,NaN


In [11]:
#drop na values from columns

df = df.dropna()
df

,Age,Occupation,Annual_Income,Delay_from_due_date,Num_of_Delayed_Payment,Outstanding_Debt,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Payment_Behaviour,Monthly_Balance,Credit_Score
0,23,Scientist,19114.12,3,7,809.98,22 Years and 1 Months,No,49.574949,High_spent_Small_value_payments,312.49408867943663,Good
2,-500,Scientist,19114.12,3,7,809.98,22 Years and 3 Months,No,49.574949,Low_spent_Medium_value_payments,331.2098628537912,Good
3,23,Scientist,19114.12,5,4,809.98,22 Years and 4 Months,No,49.574949,Low_spent_Small_value_payments,223.45130972736786,Good
5,23,Scientist,19114.12,8,4,809.98,22 Years and 6 Months,No,49.574949,!@9#%8,340.4792117872438,Good
6,23,Scientist,19114.12,3,8_,809.98,22 Years and 7 Months,No,49.574949,Low_spent_Small_value_payments,244.5653167062043,Good
...,...,...,...,...,...,...,...,...,...,...,...,...
99994,25,Mechanic,39628.99,20,6,502.38,31 Years and 5 Months,No,35.104023,High_spent_Medium_value_payments,410.256158,Poor
99995,25,Mechanic,39628.99,23,7,502.38,31 Years and 6 Months,No,35.104023,High_spent_Large_value_payments,479.866228,Poor
99996,25,Mechanic,39628.99,18,7,502.38,31 Years and 7 Months,No,35.104023,High_spent_Medium_value_payments,496.65161,Poor
99997,25,Mechanic,39628.99,27,6,502.38,31 Years and 8 Months,No,35.104023,High_spent_Large_value_payments,516.809083,Poor


In [16]:
for col in df.columns:
    print(col + ": ")
    print("")
    print(df[col].value_counts())
    print("")

Age: 

28      2360
38      2351
26      2342
31      2330
35      2302
        ... 
6722       1
3726       1
6111       1
6227       1
1342       1
Name: Age, Length: 1553, dtype: int64

Occupation: 

_______          5910
Lawyer           5478
Mechanic         5292
Engineer         5289
Architect        5282
Teacher          5261
Scientist        5260
Developer        5209
Accountant       5205
Entrepreneur     5175
Media_Manager    5174
Journalist       5121
Doctor           5094
Manager          4990
Musician         4975
Writer           4915
Name: Occupation, dtype: int64

Annual_Income: 

9141.63      15
40341.16     14
95596.35     14
17816.75     14
20867.67     14
             ..
24312.94_     1
69557.68_     1
91516.44_     1
41015.31_     1
39628.99_     1
Name: Annual_Income, Length: 18019, dtype: int64

Delay_from_due_date: 

 15    3045
 13    2863
 8     2818
 14    2766
 10    2753
       ... 
-4       52
 65      42
-5       27
 66      27
 67      20
Name: Delay_fro

In [ ]:
#from the above we can see that we need to change some noise within the columns (values that dont makes sense)